In [25]:
import importlib
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from CASutils import filter_utils as filt
from CASutils import readdata_utils as read
from CASutils import calendar_utils as cal
import sys
from datetime import timedelta, datetime
import pandas as pd

importlib.reload(filt)
importlib.reload(read)
importlib.reload(cal)

<module 'CASutils.calendar_utils' from '/home/islas/python/CASanalysis/CASutils/calendar_utils.py'>

In [2]:
ystart=1979 ; yend=2014 ; nyears=yend-ystart+1

In [3]:
dat1970 = xr.open_dataset("/project/haggis/obs/BEST_daily/Complete_TAVG_Daily_LatLong1_1970.nc")
dat1980 = xr.open_dataset("/project/haggis/obs/BEST_daily/Complete_TAVG_Daily_LatLong1_1980.nc")
dat1990 = xr.open_dataset("/project/haggis/obs/BEST_daily/Complete_TAVG_Daily_LatLong1_1990.nc")
dat2000 = xr.open_dataset("/project/haggis/obs/BEST_daily/Complete_TAVG_Daily_LatLong1_2000.nc")
dat2010 = xr.open_dataset("/project/haggis/obs/BEST_daily/Complete_TAVG_Daily_LatLong1_2010.nc")

In [4]:
# convert the date_number into a useful time axis
time = cal.fracofyear2date(np.array(dat1970.date_number))
dat1970 = dat1970.assign_coords(time=time)

time = cal.fracofyear2date(np.array(dat1980.date_number))
dat1980 = dat1980.assign_coords(time=time)

time = cal.fracofyear2date(np.array(dat1990.date_number))
dat1990 = dat1990.assign_coords(time=time)

time = cal.fracofyear2date(np.array(dat2000.date_number))
dat2000 = dat2000.assign_coords(time=time)

time = cal.fracofyear2date(np.array(dat2010.date_number))
dat2010 = dat2010.assign_coords(time=time)

In [5]:
# concatenate
best = xr.concat([dat1970, dat1980, dat1990, dat2000, dat2010], "time", data_vars=["temperature"])

In [6]:
# remove Feb 29ths
best = best.sel(time=~((best.time.dt.month==2) & (best.time.dt.day == 29)))
best.time.encoding['calendar'] = "noleap"

In [7]:
# grab out 1979-2014
best = best.sel(time=slice("1979-01-01","2014-12-31"))

In [9]:
# repeat the climatology, assign appropriate coords and add to anomalies
#clim = np.repeat( np.array(best.climatology), nyears, axis=0)
clim = np.tile(best.climatology, (nyears, 1,1))
clim_xr = xr.DataArray(clim, dims = best.temperature.dims, coords = best.temperature.coords)
best['temperature'] = best['temperature']+clim_xr

In [26]:
# making day string for grouping since we have removed leap years
daystr = xr.DataArray(best.indexes['time'].strftime('%m-%d'), coords = best.time.coords, name="daystr")
bestseason = best.temperature.groupby(daystr).mean('time')
trefht4harm = filt.calc_season_nharm(bestseason, 4, dimtime=0)
anoms = best.temperature.groupby(daystr)-trefht4harm
djfanoms = cal.group_season_daily(anoms,'DJF')
djfmean = djfanoms.mean('day')
djfanoms = djfanoms - djfmean

you got nans


In [40]:
bestvar = np.var(djfanoms, axis=(0,1))
bestvar = bestvar.rename('bestvar')

/project/cas/islas/miniconda3/envs/ecpaperenv/lib/python3.6/site-packages/xarray/core/nputils.py:227: RuntimeWarning: Degrees of freedom <= 0 for slice.
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)


In [41]:
bestvar.to_netcdf(path="/project/cas/islas/savs/python/snowpaper/DATA_SORT/deseasonalized_tvar/TVAR_BEST.nc")